# Transfer Learning on Oxford Flowers 102

This notebook demonstrates how to fine-tune a pretrained **MobileNetV2** (ImageNet) on the **Oxford Flowers 102** dataset using PyTorch. We also compare different data augmentation strategies and their effect on accuracy.


In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

## Dataset

In [ ]:
from torchvision.datasets import Flowers102

train_data_raw = Flowers102(
    root="data",
    split="train",
    transform=None,
    download=True
)

val_data_raw = Flowers102(
    root="data",
    split="val",
    transform=None,
    download=True
)

test_data_raw = Flowers102(
    root="data",
    split="test",
    transform=None,
    download=True
)

print(f"Train size: {len(train_data_raw)}")
print(f"Val size: {len(val_data_raw)}")
print(f"Test size: {len(test_data_raw)}")
print(f"\nNumber of classes: {len(set(train_data_raw._labels))}")

In [ ]:
def show_random_image():
    idx = random.randint(0, len(train_data_raw) - 1)
    img, label = train_data_raw[idx]
    
    plt.figure(figsize=(4, 4))
    plt.imshow(img)
    plt.title(f'Class {label}')
    plt.axis('off')
    plt.show()

show_random_image()

## Data Transforms and Augmentations

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
train_data = Flowers102(root="data", split="train", transform=train_transform, download=True)
val_data = Flowers102(root="data", split="val", transform=val_test_transform, download=True)
test_data = Flowers102(root="data", split="test", transform=val_test_transform, download=True)

batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}, Test batches: {len(test_loader)}")

imgs, lbls = next(iter(train_loader))
print(f"Batch: {imgs.shape}, Labels: {lbls.shape}")

## Pretrained MobileNetV2

In [ ]:
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

In [ ]:
num_classes = 102
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)



print(model.classifier)


total = sum(p.numel() for p in model.parameters())
print(f"Total params: {total}")

## Training Setup

In [ ]:
def get_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in loader:
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return correct / total

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for imgs, labels in loader:
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * labels.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    return total_loss / total, correct / total

## Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

num_epochs = 10
best_val = 0.0

train_losses = []
train_accs = []
val_accs = []

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer)
    val_acc = get_accuracy(model, val_loader)
    
    scheduler.step()
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    
    if val_acc > best_val:
        best_val = val_acc

    
    lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch} | Loss: {train_loss:.4f} | Train: {train_acc:.4f} | Val: {val_acc:.4f} | LR: {lr:.6f}")

print(f"Best val accuracy: {best_val:.4f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(train_losses)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.grid(True)

ax2.plot(train_accs, label='Train')
ax2.plot(val_accs, label='Val')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Test Set Evaluation

In [ ]:
test_acc = get_accuracy(model, test_loader)
print(f"Test accuracy: {test_acc:.4f}")

## Example Predictions

In [ ]:
model.eval()

correct = []
incorrect = []

indices = list(range(len(test_data)))
random.shuffle(indices)

for idx in indices:
    if len(correct) >= 5 and len(incorrect) >= 5:
        break
        
    img, label = test_data_raw[idx]
    img_t, _ = test_data[idx]
    
    with torch.no_grad():
        pred = model(img_t.unsqueeze(0)).argmax(1).item()
    
    if pred == label and len(correct) < 5:
        correct.append((img, label, pred))
    elif pred != label and len(incorrect) < 5:
        incorrect.append((img, label, pred))

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, (img, label, pred) in enumerate(correct):
    axes[0, i].imshow(img)
    axes[0, i].set_title(f'{label} | {pred}', color='green')
    axes[0, i].axis('off')

for i, (img, label, pred) in enumerate(incorrect):
    axes[1, i].imshow(img)
    axes[1, i].set_title(f'{label} | {pred}', color='red')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()


## Augmentation Experiments

Let's compare the baseline augmentation pipeline against a more aggressive one that adds vertical flips, random rotation, and stronger Gaussian blur.

In [ ]:
advanced_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 3.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

train_data_adv = Flowers102(root="data", split="train", transform=advanced_transform, download=True)
train_loader_adv = DataLoader(train_data_adv, batch_size=32, shuffle=True)


In [ ]:
model_aug = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
model_aug.classifier[1] = nn.Linear(model_aug.classifier[1].in_features, 102)

criterion_aug = nn.CrossEntropyLoss()
optimizer_aug = optim.Adam(model_aug.parameters(), lr=1e-3)
scheduler_aug = optim.lr_scheduler.StepLR(optimizer_aug, step_size=30, gamma=0.1)

num_epochs_adv = 5
train_losses_adv = []
train_accs_adv = []
val_accs_adv = []

for epoch in range(1, num_epochs_adv + 1):
    train_loss, train_acc = train_one_epoch(model_aug, train_loader_adv, criterion_aug, optimizer_aug)
    val_acc = get_accuracy(model_aug, val_loader)

    scheduler_aug.step()

    train_losses_adv.append(train_loss)
    train_accs_adv.append(train_acc)
    val_accs_adv.append(val_acc)

    lr = optimizer_aug.param_groups[0]['lr']
    print(f"Epoch {epoch} | Loss: {train_loss:.4f} | Train: {train_acc:.4f} | Val: {val_acc:.4f} | LR: {lr:.6f}")

test_acc_aug = get_accuracy(model_aug, test_loader)
print(f"\nAdvanced augmentation test accuracy: {test_acc_aug:.4f}")
print(f"Original test accuracy: {test_acc:.4f}")
print(f"Difference: {test_acc_aug - test_acc:.4f}")


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(train_losses, label='Original')
ax1.plot(train_losses_adv, label='Advanced Aug')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss Comparison')
ax1.legend()
ax1.grid(True)

ax2.plot(val_accs, label='Original')
ax2.plot(val_accs_adv, label='Advanced Aug')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Validation Accuracy Comparison')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()
